In [ ]:
# INITIALIZATION
# INIT : IMPORTS
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)
import math
import requests
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from pandasgui import show
import ipywidgets as widgets
from IPython.display import display
import string
import datetime

In [ ]:
# INPUT API ID + TOKEN 
xdrauthid = input()
token = input()
myurl = input()

In [ ]:
# INIT : API 1/2
headers = {
  'Content-Type': 'application/json',
  'x-xdr-auth-id': xdrauthid,
  'Authorization': token
}

# DEBUG 
# print(headers)

In [ ]:
# INIT : API 2/2
parameters = {}

resp1 = requests.post(url=myurl,
            headers=headers,
            json=parameters)

# Testing that the API key is valid
if resp1.status_code == 200:
    print("ACCES API XDR OK. RETURN CODE: " + str(resp1.status_code))
else:
    print("ACCES API XDR KO. RETURN CODE: " + str(resp1.status_code))

In [ ]:
# INPUT CSV FILE
# warning : expect the header 'endpoint_name' when no FQDN, no header otherwise
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

df1 = pd.read_csv(filename,names=['endpoint_name'])

In [ ]:
# OPTIONAL : removing the FQDN
df1[['endpoint_name', 'domain']] = df1['endpoint_name'].str.split('.', 1, expand=True)
print(df1)

In [ ]:
# FAST | GET ENDPOINTS INFO OVER A LIST < 100
url = myurl + "public_api/v1/endpoints/get_endpoint/"

payload = json.dumps({
   "request_data":{
      "sort":{
         "field":"endpoint_id",
         "keyword":"ASC"
      },
      "filters":[
         {
            "field":"hostname",
            "operator":"in",
            "value":[""]
         },
         #{
         #   "field":"endpoint_status",
         #   "operator":"in",
         #   "value":[
         #       "connected"
         #   ]
         #},
      ]
   }
})

pp = json.loads(payload)
pp['request_data']['filters'][0]['value'] = np.array(df1['endpoint_name'].values).tolist()
pp = json.dumps(pp)
resp2 = requests.request("POST", url, headers=headers, data=pp)

jsonResponse = resp2.json()

# Convert epoch timestamps + Display Pandas GUI 
def milli_epoch_to_datetime(epoch):
    s = epoch / 1000.0
    return datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f')

i=0
for ep in jsonResponse['reply']['endpoints']:
    jsonResponse['reply']['endpoints'][i]['first_seen'] = milli_epoch_to_datetime(ep['first_seen'])
    jsonResponse['reply']['endpoints'][i]['last_seen']  = milli_epoch_to_datetime(ep['last_seen'])
    i=i+1

df2 = pd.json_normalize(jsonResponse, record_path=['reply',['endpoints']])
gui = show(df2)


In [ ]:
# SLOW | GET ENDPOINTS INFO OVER A LIST > 100

url = myurl + "public_api/v1/endpoints/get_endpoint/"

# rlen MUST BE < 100, here it collects endpoints data by bunch of 80
rlen = 80
xround = math.ceil(len(df1)/rlen)
r=0; rl1=0; rl2=rlen-1;
df2 = [] 
jsonResponse = {}

# Convert epoch timestamps + Display Pandas GUI 
def milli_epoch_to_datetime(epoch):
    s = epoch / 1000.0
    return datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f')

while r < xround: 

    print("Round ", r, ". Getting endpoints from ", rl1, " to ",rl2,".")
    payload = json.dumps({
     "request_data":{
     "sort":{
        "field":"endpoint_id",
        "keyword":"ASC"
    },
    "filters":[
    {
       "field":"hostname",
       "operator":"in",
       "value":[""]
    },
          #{
            #   "field":"endpoint_status",
            #   "operator":"in",
            #   "value":[
            #       "connected"
            #   ]
            #},
    ]
    }
    })
      
    pp = json.loads(payload)
    dff1 = df1.loc[rl1:rl2]
    pp['request_data']['filters'][0]['value'] = np.array(dff1['endpoint_name'].values).tolist()
    pp = json.dumps(pp)
    resp2 = requests.request("POST", url, headers=headers, data=pp)
    jsonResponse = resp2.json()

    if r == 0:
            df2 = pd.json_normalize(jsonResponse, record_path=['reply',['endpoints']])
    else:
            df2 = df2.append(pd.json_normalize(jsonResponse, record_path=['reply',['endpoints']]),ignore_index=True)
    
    r = r + 1
    rl1 = rl2+1
    rl2 = rl1+rlen-1


gui = show(df2)

In [ ]:
# DEBUG 
# print(pp)
# print(jsonResponse)

In [ ]:
# DISPLAY HOSTS WITH XDR
# i=0
#for ep in df2.iterrows:
#   df2['first_seen'] = milli_epoch_to_datetime(ep['first_seen'])
#   df2['last_seen']  = milli_epoch_to_datetime(ep['last_seen'])
#   i=i+1

gui = show(df2)

In [ ]:
# DISPLAY HOSTS WITHOUT XDR
df3 = pd.concat([df1['endpoint_name'],df2['endpoint_name']]).drop_duplicates(keep=False)
gui = show(df3)